In [ ]:
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
import pickle
import os

from operator import add
import copy

In [ ]:
#Connect to drive to import files
from google.colab import drive
drive.mount('/content/drive')
#drive.mount('/gdrive', force_remount=True)

Mounted at /content/drive


# Activations w-scores

In [ ]:
cols_to_move = [ 'fullsid',	'grp',	'sex1f'	,'education_y'	,'age'	,'MRI_field_strength'	,'eTIV', 'MMSE' ]
cols_to_move2 = [ 'sid', 'sample', 'fullsid',	'grp',	'sex1f'	,'education_y'	,'age'	,'MRI_field_strength'	,'eTIV', 'MMSE' ]


#Volume w-score file

In [ ]:
def filter_and_rename_columns(df, suffix):
    """
    Given a suffix, find all columns in a pandas DataFrame that have that suffix.
    Slice the DataFrame with these chosen columns and remove the suffix from column names.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        suffix (str): The suffix to filter column names.

    Returns:
        pd.DataFrame: A new DataFrame with selected columns and renamed column names.
    """
    # Find columns that end with the given suffix
    selected_cols = [col for col in df.columns if col.endswith(suffix)]

    # Slice the DataFrame with these columns
    df_subset = df[selected_cols].copy()

    # Rename columns by removing the suffix
    df_subset.columns = [col[:-len(suffix)] for col in selected_cols]

    return df_subset

In [ ]:
df_w = pd.read_csv( '/content/drive/My Drive/ontology/Rel_Vol_cortThk_w-score-ROIs_1_density_total.csv', index_col=False)

<ipython-input-5-e26b5164d041>:1: DtypeWarning: Columns (105,242,272) have mixed types. Specify dtype option on import or set low_memory=False.
  df_w = pd.read_csv( '/content/drive/My Drive/ontology/Rel_Vol_cortThk_w-score-ROIs_1_density_total.csv', index_col=False)


In [ ]:
df_w_vol = filter_and_rename_columns(df_w, '_vol')   #_rel, _cortThk
df_w_vol['fullsid'] = df_w['fullsid']


df_w_act = filter_and_rename_columns(df_w, '_rel')   #_rel, _cortThk
df_w_act['fullsid'] = df_w['fullsid']


df_w_cortThk = filter_and_rename_columns(df_w, '_cortThk')   #_rel, _cortThk
df_w_cortThk['fullsid'] = df_w['fullsid']

In [ ]:
list_ROIs = list(set(df_w_vol.columns.to_list()) - set(cols_to_move2))
empty_cortThk_columns = [col for col in df_w_cortThk.columns if df_w_cortThk[col].isna().all()]


#TextualExpaliantion

In [ ]:
# Dataframe with which scores z-scores/w-scores for an AD-sample;  it has the parent-child relations built into it.
# And the ROIs are manually ordered and have an associated (ring) 'level' with them,
df_ = pd.read_csv('/content/drive/My Drive/ontology/vis_test.csv',index_col=False).fillna('')
child_parent_dict = dict(zip(df_.ROI,df_.parent))     #A dictionary, mapping all the child->parent ROI
ROI_level_dict = dict(zip(df_.ROI,df_.level))         #A dictionary, mapping all the child->level (i.e, distance from the root)

In [ ]:
#The main aggregation algo :)
def recurrsive_ele(ROI_list, level):
  #An algo to remove all the child nodes
  #if, parents nodes are also deviating more than the threshold (i.e. are found to be of interest)

  if level < 4:
    return ROI_list
 # if len(ROI_list)==1:
  #  return ROI_list

  else:
    ROI_list_copy = copy.deepcopy(ROI_list)
    for ROI in ROI_list:
      if (ROI_level_dict[ROI] == level) & (child_parent_dict[ROI] in ROI_list):
        ROI_list_copy.remove(ROI)
    ROI_list_copy = recurrsive_ele(ROI_list_copy, level-1)  #Go to the previous/upper level of parents
    return ROI_list_copy

In [ ]:
threshold = 2

flip = -1
# [1,-1]
#  -1 > when the AD activation distribution is lower than CN activation distribution
#   1 > when the AD activation distribution is higer than CN activation distribution
#This denepds on the node of DL model used for creating the activations


In [ ]:
list_represtatives = [  #--------AD----------
                        '0371_AIBL',
                      ]

In [ ]:
# %%capture cap
#for i in list_represtatives:
from pprint import pprint

def genereate_report(id):
  i = id

  row_act = df_w_act[df_w_act['fullsid'] == i]
  row_vol = df_w_vol[df_w_vol['fullsid'] == i]
  row_cortThk = df_w_cortThk[df_w_cortThk['fullsid'] == i]

  print('\n')
  print('---------------------------  NEW SAMPLE  --------------------------')
  print(row_act.fullsid)

  activation_focused_ROIs = []
  volume_focused_ROIs = []
  corticalThickness_focused_ROIs = []


  #find all the ROIs of interest which were found usefull by either the activation or volumetric method
  for ROI in list_ROIs:   #df_w_act.columns[10:].to_list(), 9,10

    #Unidirection. For activation, we only condiser if they are more than a certain threshold in one direction.
    #Heuristic#1 Threshold.
    #Heuristic#2 The unidirection nature. Observed from mean ad 'activation' sunmmary plot for 'node=1'
    if (row_act[ROI].values[0] < flip*threshold):                       #.values[0]
      activation_focused_ROIs.append(ROI)

    #bidirectional. For volumetric w-score, we are interesting in both atrophy and expansion patterns
    if (row_vol[ROI].values[0] < flip*threshold) or (row_vol[ROI].values[0] > threshold):
      volume_focused_ROIs.append(ROI)

    #Unidirection. For corticalThickness, we only condiser if they are less than a certain threshold.
    #As unlike ventricle volume, there is no pathologic usecase to expect corticalThickness to be more than 2-wscore
    if (row_cortThk[ROI].values[0] < flip*threshold):                       #.values[0]
      corticalThickness_focused_ROIs.append(ROI)

  #LIST of ROI over which both the methods - volumetry and CNN relevances, agree upon.
  agree3_intersect = set(activation_focused_ROIs) & set(volume_focused_ROIs) & set(corticalThickness_focused_ROIs)
  agree2_intersect = set(activation_focused_ROIs) & set(volume_focused_ROIs)
  ROI_intersect_list = list(agree3_intersect)
  for ele in list(agree2_intersect-agree3_intersect ):
      if ele in empty_cortThk_columns:        #these were the ROIs which were removed because Cortical Thickness measures in these regions are not calculated (NAN)
          ROI_intersect_list.append(ele)      #So we add them back into the pool of ROIs we consider of reporting



  verbose=False
  if verbose:
    #print the full list of ROIs found to be interest.
    ROI_list = ROI_intersect_list
    ROI_list_print = [text.replace('_', ' ') for text in ROI_list]
    print('Pathologic ROIs')
    pprint(', '.join(ROI_list_print))
    print('\n')
  else:
    #remove the child ROI, for which it's parent were also found to be of interest
    ROI_list = recurrsive_ele(ROI_intersect_list,4)
    ROI_intersect_list_print = [text.replace('_', ' ') for text in ROI_intersect_list]
    ROI_list_print = [text.replace('_', ' ') for text in ROI_list]
    print('Pathologic ROIs')
    print(', '.join(ROI_intersect_list_print))
    print('\n')
    print('Pathologic ROIs presented to the user')
    print(', '.join(ROI_list_print))
    print('\n')

  print('-- Dynamic Result [Numeric Report]: Volumetry + Cortical Thickness + CNN Relevance --')
  for ROI in ROI_list:
    if ROI in empty_cortThk_columns:
      print('Volumetric {} (w-score)            : '.format(ROI.replace('_',' ')) + '{0:.2f}'.format(row_vol[ROI].values[0]))
      #print('Cortical Thickness not measured for {}: '.format(ROI.replace('_',' ')))
      print('Deep Learning activity {} (w-score): '.format(ROI.replace('_',' ')) + '{0:.2f}'.format(row_act[ROI].values[0]))
    else:
      print('Volumetric {} (w-score)            : '.format(ROI.replace('_',' ')) + '{0:.2f}'.format(row_vol[ROI].values[0]))
      print('Cortical Thickness {} (w-score)    : '.format(ROI.replace('_',' ')) + '{0:.2f}'.format(row_cortThk[ROI].values[0]))
      print('Deep Learning activity {} (w-score): '.format(ROI.replace('_',' ')) + '{0:.2f}'.format(row_act[ROI].values[0]))
    print('\n')


  print('-- Dynamic Result [Textual]: Volumetry + Cortical Thickness + CNN Relevance --')
  for ROI in ROI_list:
    avg = 0
    if ROI in empty_cortThk_columns:
      list_ = np.abs([row_vol[ROI].values[0], row_act[ROI].values[0]])
    else:
      list_ = np.abs([row_vol[ROI].values[0], row_act[ROI].values[0], row_cortThk[ROI].values[0]])
    avg = sum(list_) / len(list_)

    keyword=''
    if avg > 4:
      keyword = 'Strong'
    elif avg > 3:
      keyword = 'Moderate'
    elif avg > 2:
      keyword = 'Mild'

    flag=''
    if row_vol[ROI].values[0] < 0:
      flag='atrophied'
    else:
      flag='enlarged'

    print('{} pathology in {} {}'.format(keyword, flag, ROI.replace('_',' ')) + '\n' +' (volume w-score: ' + '{0:.2f}'.format(row_vol[ROI].values[0]) + ', relevance w-score: ' +  '{0:.2f}'.format(row_act[ROI].values[0]) + ')' )
    print('\n')

 ###----STATIC PRECODED PRESENTATION OF LIMITED REGIONS
  df = pd.DataFrame(dict(
    vol=[       row_vol['Right_Hippocampus'].values[0], row_vol['Left_Hippocampus'].values[0],
                row_vol['Right_Frontal_Lobe'].values[0], row_vol['Left_Frontal_Lobe'].values[0],
                row_vol['Right_Parietal_Lobe'].values[0], row_vol['Left_Parietal_Lobe'].values[0],
                row_vol['Right_Temporal_Lobe'].values[0], row_vol['Left_Temporal_Lobe'].values[0],
                row_vol['Right_Occipital_Lobe'].values[0], row_vol['Left_Occipital_Lobe'].values[0],
                row_vol['Brain_Region'].values[0],


    ],
    act=[       row_act['Right_Hippocampus'].values[0], row_act['Left_Hippocampus'].values[0],
                row_act['Right_Frontal_Lobe'].values[0], row_act['Left_Frontal_Lobe'].values[0],
                row_act['Right_Parietal_Lobe'].values[0], row_act['Left_Parietal_Lobe'].values[0],
                row_act['Right_Temporal_Lobe'].values[0], row_act['Left_Temporal_Lobe'].values[0],
                row_act['Right_Occipital_Lobe'].values[0], row_act['Left_Occipital_Lobe'].values[0],
                row_act['Brain_Region'].values[0],

    ],

    cortThk=[       row_cortThk['Right_Hippocampus'].values[0], row_cortThk['Left_Hippocampus'].values[0],
                row_cortThk['Right_Frontal_Lobe'].values[0], row_cortThk['Left_Frontal_Lobe'].values[0],
                row_cortThk['Right_Parietal_Lobe'].values[0], row_cortThk['Left_Parietal_Lobe'].values[0],
                row_cortThk['Right_Temporal_Lobe'].values[0], row_cortThk['Left_Temporal_Lobe'].values[0],
                row_cortThk['Right_Occipital_Lobe'].values[0], row_cortThk['Left_Occipital_Lobe'].values[0],
                row_cortThk['Brain_Region'].values[0],

    ],

    theta=[     'Right Hippocampal', 'Left Hippocampal',
                'Right Frontal Lobe', 'Left Frontal Lobe',
                'Right Parietal Lobe', 'Left Parietal Lobe',
                'Right Temporal Lobe', 'Left Temporal Lobe',
                'Right Occipital Lobe', 'Left Occipital Lobe',
                'Whole Brain',

    ]
           ))



  fig = go.Figure()

  fig.add_trace(
      go.Scatterpolar(
          r=df['vol'],
          theta=df['theta'],
          name='Volumetric w-score'
      )
  )
  fig.add_trace(
      go.Scatterpolar(
          r=df['act'],
          theta=df['theta'],
          name='CNN attribution w-score'

      )
  )
  fig.add_trace(
      go.Scatterpolar(
          r=df['cortThk'],
          theta=df['theta'],
          name='Cortical Thickess w-score'

      )
  )


  fig.update_layout(
    polar=dict(
      radialaxis=dict(
        visible=True,
        range=[1, -5]   ##TO control the inversion of axis
      )),
    showlegend=True
  )
  fig.show()


In [ ]:
genereate_report('0371_AIBL')



---------------------------  NEW SAMPLE  --------------------------
228    0371_AIBL
Name: fullsid, dtype: object
Pathologic ROIs
Left_Inferior_Temporal, Left_Middle_Temporal, Left_Temporal_Lobe, Left_Superior_Temporal, Right_Inferior_Temporal, Left_Inf-Lat-Vent


Pathologic ROIs presented to the user
Left_Temporal_Lobe, Right_Inferior_Temporal, Left_Inf-Lat-Vent


-- Dynamic Result [Numeric Report]: Volumetry + Cortical Thickness + CNN Relevance --
Volumetric Left Temporal Lobe (w-score)            : -2.86
Cortical Thickness Left Temporal Lobe (w-score)    : -2.95
Deep Learning activity Left Temporal Lobe (w-score): -4.69


Volumetric Right Inferior Temporal (w-score)            : -2.21
Cortical Thickness Right Inferior Temporal (w-score)    : -2.20
Deep Learning activity Right Inferior Temporal (w-score): -2.65


Volumetric Left Inf-Lat-Vent (w-score)            : 3.02
Deep Learning activity Left Inf-Lat-Vent (w-score): -2.39


-- Dynamic Result [Textual]: Volumetry + Cortical Thic